In [ ]:
import pandas as pd
import os

# Mapping dictionary
name_to_id = {
    "Ethel": "24WMR08861",
    "Jesmine": "24WMR08866",
    "Khaili": "24WMR08863",
    "Kit": "24WMR08870"
}

# Reverse mapping: ID → Name
id_to_name = {v: k for k, v in name_to_id.items()}

def check_accuracy(dataset, detector, test_dir="test_photos", threshold=0.5):
    results = []
    total = 0
    correct = 0

    for person_folder in os.listdir(test_dir):
        person_path = os.path.join(test_dir, person_folder)
        if not os.path.isdir(person_path):
            continue

        for filename in os.listdir(person_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(person_path, filename)
                img = cv.imread(img_path)
                rgb_img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

                faces = detector.get(rgb_img)
                if not faces:
                    results.append([
                        filename, "-", "-", "-", 
                        person_folder, name_to_id.get(person_folder, person_folder), "❌"
                    ])
                    continue

                # pick highest confidence face
                faces.sort(key=lambda f: f.det_score, reverse=True)
                best_face = faces[0]

                # align + preprocess
                aligned_face = align_face(rgb_img, best_face)
                enhanced_face = enhance_contrast(aligned_face)

                # get embedding
                embedding = get_face_embedding_from_obj(best_face)

                # recognize
                person_id, pred_name, score = recognize_face(embedding, dataset, threshold=threshold)

                total += 1
                ground_truth_name = person_folder
                ground_truth_id = name_to_id.get(person_folder, person_folder)

                pred_id = None
                pred_person_name = None
                if pred_name:
                    pred_id = os.path.splitext(pred_name)[0]  # remove .jpg
                    pred_person_name = id_to_name.get(pred_id, "Unknown")

                # ✅ Compare by ID
                if pred_id == ground_truth_id:
                    correct += 1
                    status = "✅"
                else:
                    status = "❌"

                results.append([
                    filename, 
                    pred_person_name if pred_person_name else "Unknown", 
                    pred_id if pred_id else "Unknown",
                    f"{score:.2f}", 
                    ground_truth_name, 
                    ground_truth_id, 
                    status
                ])

    accuracy = correct / total if total > 0 else 0
    print("\nFinal Accuracy: {:.2f}% ({}/{})\n".format(accuracy*100, correct, total))

    # Convert results to DataFrame for table printing
    df = pd.DataFrame(
        results, 
        columns=["Image", "Predicted Name", "Predicted ID", "Score", "Ground Truth Name", "Ground Truth ID", "Result"]
    )
    print(df.to_string(index=False))

    return df


# --- Usage ---
df = check_accuracy(
    embeddings_data,   # dataset loaded from pickle
    detector,          # your initialized face detector
    test_dir="test_photos",  # path to your test images
    threshold=0.5      # similarity threshold
)


In [ ]:
import pandas as pd
import cv2
import os
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Mapping dictionary ---
name_to_id = {
    "Ethel": "24WMR08861",
    "Jesmine": "24WMR08866",
    "Khaili": "24WMR08863",
    "Kit": "24WMR08870"
}
id_to_name = {v: k for k, v in name_to_id.items()}

# --- Helper: get embedding from BGR image ---
def get_face_embedding_from_image(img):
    results = detector.detect_faces(img)
    if len(results) == 0:
        return None
    
    face = results[0]
    x, y, w, h = face['box']
    x, y = max(0, x), max(0, y)
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (160, 160))
    
    embedding = embedder.embeddings([face_img])[0]
    return l2_normalize(embedding)

# --- Accuracy function ---
def check_accuracy(database, test_dir="test_photos", threshold=0.5):
    results = []
    total = 0
    correct = 0

    for person_folder in os.listdir(test_dir):
        person_path = os.path.join(test_dir, person_folder)
        if not os.path.isdir(person_path):
            continue

        for filename in os.listdir(person_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(person_path, filename)
                img = cv2.imread(img_path)
                if img is None:
                    continue

                embedding = get_face_embedding_from_image(img)
                if embedding is None:
                    results.append([filename, "-", "-", "-", person_folder, name_to_id.get(person_folder, person_folder), "❌"])
                    continue

                # Compare with database
                similarities = []
                for db_name, db_emb in database.items():
                    sim_score = cosine_similarity([embedding], [db_emb])[0][0]
                    similarities.append((db_name, sim_score))

                similarities.sort(key=lambda x: x[1], reverse=True)
                best_match, best_score = similarities[0]

                pred_id = best_match
                pred_name = id_to_name.get(pred_id, "Unknown")
                ground_truth_name = person_folder
                ground_truth_id = name_to_id.get(person_folder, person_folder)

                total += 1
                if best_score >= threshold and pred_id == ground_truth_id:
                    correct += 1
                    status = "✅"
                else:
                    status = "❌"

                results.append([
                    filename,
                    pred_name,
                    pred_id,
                    f"{best_score:.2f}",
                    ground_truth_name,
                    ground_truth_id,
                    status
                ])

    accuracy = correct / total if total > 0 else 0
    print("\nFinal Accuracy: {:.2f}% ({}/{})\n".format(accuracy*100, correct, total))

    df = pd.DataFrame(results, columns=[
        "Image", "Predicted Name", "Predicted ID", "Score", "Ground Truth Name", "Ground Truth ID", "Result"
    ])
    print(df.to_string(index=False))
    return df

df = check_accuracy(face_database, test_dir="test_photos", threshold=0.5)

In [ ]:
import pandas as pd
import cv2
import os
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# --- Mapping dictionary ---
name_to_id = {
    "Ethel": "24WMR08861",
    "Jesmine": "24WMR08866",
    "Khaili": "24WMR08863",
    "Kit": "24WMR08870"
}
id_to_name = {v: k for k, v in name_to_id.items()}


# -----------------------------
# Function 1: InsightFace embedding extractor
# -----------------------------
def insightface_embedding(img, detector, embedder):
    """Extract face embedding using InsightFace pipeline"""
    faces = detector.get(img)
    if not faces:
        return None
    
    # pick highest confidence face
    faces.sort(key=lambda f: f.det_score, reverse=True)
    best_face = faces[0]

    # align face (assuming you already have align_face + enhance_contrast)
    aligned_face = align_face(img, best_face)
    enhanced_face = enhance_contrast(aligned_face)

    # get embedding
    embedding = embedder.embeddings([enhanced_face])[0]
    return embedding / np.linalg.norm(embedding)


# -----------------------------
# Function 2: MTCNN embedding extractor
# -----------------------------
def mtcnn_embedding(img, detector, embedder):
    """Extract face embedding using MTCNN + FaceNet pipeline"""
    results = detector.detect_faces(img)
    if len(results) == 0:
        return None
    
    face = results[0]
    x, y, w, h = face['box']
    x, y = max(0, x), max(0, y)
    face_img = img[y:y+h, x:x+w]
    face_img = cv2.resize(face_img, (160, 160))

    embedding = embedder.embeddings([face_img])[0]
    return embedding / np.linalg.norm(embedding)


# -----------------------------
# Function 3: Accuracy Checker
# -----------------------------
def check_accuracy(database, test_dir, embed_func, detector, embedder, threshold=0.5):
    """
    database   : dict {id: embedding}
    test_dir   : path to test images
    embed_func : embedding function (insightface_embedding OR mtcnn_embedding)
    detector   : detector object (MTCNN or InsightFace)
    embedder   : embedding model (FaceNet or InsightFace embedder)
    """
    results = []
    total, correct = 0, 0

    for person_folder in os.listdir(test_dir):
        person_path = os.path.join(test_dir, person_folder)
        if not os.path.isdir(person_path):
            continue

        for filename in os.listdir(person_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(person_path, filename)
                img = cv2.imread(img_path)
                if img is None:
                    continue

                rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                embedding = embed_func(rgb_img, detector, embedder)

                if embedding is None:
                    results.append([filename, "-", "-", "-", person_folder,
                                    name_to_id.get(person_folder, person_folder), "❌"])
                    continue

                # Compare with database
                similarities = [(db_id, cosine_similarity([embedding], [db_emb])[0][0])
                                for db_id, db_emb in database.items()]
                similarities.sort(key=lambda x: x[1], reverse=True)
                best_match, best_score = similarities[0]

                pred_id = best_match
                pred_name = id_to_name.get(pred_id, "Unknown")
                ground_truth_name = person_folder
                ground_truth_id = name_to_id.get(person_folder, person_folder)

                total += 1
                if best_score >= threshold and pred_id == ground_truth_id:
                    correct += 1
                    status = "✅"
                else:
                    status = "❌"

                results.append([filename, pred_name, pred_id, f"{best_score:.2f}",
                                ground_truth_name, ground_truth_id, status])

    accuracy = correct / total if total > 0 else 0
    print("\nFinal Accuracy: {:.2f}% ({}/{})\n".format(accuracy*100, correct, total))

    df = pd.DataFrame(results, columns=[
        "Image", "Predicted Name", "Predicted ID", "Score",
        "Ground Truth Name", "Ground Truth ID", "Result"
    ])
    print(df.to_string(index=False))
    return df


In [ ]:
df = check_accuracy(
    database=face_database,
    test_dir="test_photos",
    embed_func=insightface_embedding,
    detector=insightface_detector,
    embedder=insightface_embedder,
    threshold=0.5
)


In [ ]:
df = check_accuracy(
    database=face_database,
    test_dir="test_photos",
    embed_func=mtcnn_embedding,
    detector=mtcnn_detector,
    embedder=facenet_embedder,
    threshold=0.5
)
